In [1]:
import cox 
from cox.store import Store
from cox.readers import CollectionReader 
import seaborn as sns

import os 
# set environment variable so that stores can create output files
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

In [4]:
reader = CollectionReader('/home/gridsan/stefanou/cifar10/resnet18')

100%|██████████| 8/8 [00:00<00:00, 33.37it/s]


In [5]:
reader.df('logs')

,epoch,val_prec1,val_loss,train_prec1,train_loss,time,exp_id
0,1,0.0,5.648901,0.0,10.126699,21.92421,abe5e1aa-bd5c-45da-a519-d780dc9d0d4d
